In [27]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import ast
import math
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
def elimina_alfa(lista):
  lista_valori = lista.split(",")
  values = []
  for val in lista_valori:
    temp = ""
    for v in val:
      if v.isdigit():
        temp = temp+v
    values.append(int(temp))
  return sum(values)

In [29]:
def plot_confusion_matrix(conf_matrix, class_names, path):
    plt.figure(figsize=(5, 5))
    sns.heatmap(
        conf_matrix, annot=True, fmt="d", cmap="Blues", linewidths=.5,
        square=True, xticklabels=class_names, yticklabels=class_names
    )
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(path)


In [30]:
def somma_val(lista):
  return sum(ast.literal_eval(lista))

In [64]:
# def fix_dataframe(df):
#   for index, row in df.iterrows():
#     df.at[index,'Features'] = elimina_alfa(row["Features"])
#     df.at[index,'TimeInterval'] = somma_val(row["TimeInterval"])

#   return df[df["Features"]== -1 or df["TimeInterval"]== -1 ]


In [31]:
df = pd.read_csv("Event_traces.csv", sep=",")

In [32]:
df.head()

,BlockId,Label,Type,Features,TimeInterval,Latency
0,blk_-1608999687919862906,Success,NaN,"[E5,E22,E5,E5,E11,E11,E9,E9,E11,E9,E26,E26,E26...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3802
1,blk_7503483334202473044,Success,NaN,"[E5,E5,E22,E5,E11,E9,E11,E9,E11,E9,E26,E26,E26...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3802
2,blk_-3544583377289625738,Fail,21.0,"[E5,E22,E5,E5,E11,E9,E11,E9,E11,E9,E3,E26,E26,...","[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",3797
3,blk_-9073992586687739851,Success,NaN,"[E5,E22,E5,E5,E11,E9,E11,E9,E11,E9,E26,E26,E26...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",50448
4,blk_7854771516489510256,Success,NaN,"[E5,E5,E22,E5,E11,E9,E11,E9,E11,E9,E26,E26,E26...","[0.0, 0.0, 1.0, 48.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",50583


In [33]:
df_features = df[["Features","TimeInterval", "Latency"]]

In [34]:
labels_string = df[["Label"]]

In [35]:
labels = labels_string.applymap(lambda x: 1 if x == "Success" else -1)

In [ ]:
df_features[["TimeInterval"]] = df_features[["TimeInterval"]].applymap(lambda x: sum(ast.literal_eval(x)))
df_features[['Features']] = df_features[['Features']].applymap(lambda x: elimina_alfa(x))

In [15]:
# df_features = df_features.drop(41061)
# labels = labels.drop(41061)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_features, labels, test_size=0.2, random_state=42)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_train)
rf_accuracy = accuracy_score(y_train, rf_predictions)
print("Accuracy Random Forest:", rf_accuracy)
joblib.dump(rf_model, 'RF.save')

In [ ]:
anomalie_test_if = rf_model.predict(X_test)

In [ ]:
matrice_confusione = confusion_matrix(y_test, anomalie_test_if)
plot_confusion_matrix(matrice_confusione, class_names=["Anomaly","Normal"], path="rf_matrix.png")

In [ ]:
# Visualizza la matrice di confusione
print("Matrice di Confusione:")
print(matrice_confusione)

# Calcola e visualizza le metriche di classificazione
print("\nReport di Classificazione:")
print(classification_report(y_test, anomalie_test_if))